**Instituto Tecnológico de Costa Rica**

**Escuela de Ingeniería en Computación**

**Curso: Inteligencia Artificial**

**Segundo Semestre 2021**

**Profesor: Luis-Alexander Calvo-Valverde**

**Trabajo Práctico:** 04

**Datos de la entrega:** Miércoles 27 de octubre 2021, a más tardar a las 11:59 pm

**Estudiantes:**
- Eduardo Madrigal Marín
- Gabriel Vargas Rodríguez

___

# Nota Introductoria
___

Ustedes han sido contratados por la empresa FUTURA para trabajar con dos conjuntos de datos y lograr la mejor predicción posible, dadas ciertas métricas.

Para efectos de dudas o ampliaciones sobre el proyecto, además de lo que se indique en este cuaderno, considere a Luis-Alexánder Calvo-Valverde como su cliente para atender sus consultas.

___

# Parte 1  -  Clasificación (50 puntos)
___

### Considere lo siguiente:
1. Conjunto de datos: mnist_ALL.csv
1. Los atributos corresponden a una imagen de 28x28.
1. El atributo a predecir es label.
1. Proponga al menos tres algoritmos a utilizar
1. Métricas:
    1. Accuracy
    1. f1_score
1. Debe presentar en este cuaderno:
    1. Pre-procesamiento de los datos, explicando las decisiones en cada caso. Observará que en este caso viene bastante preparado
    1. Para cada uno de los algoritmos seleccionados: 
        1. Explicación del algoritmo.
        1. Explicación de la implementación seleccionada y de sus parámetros.
    1. Explicación del diseño experimental por ejecutar. Recuerde que si el algoritmo requiere seleccionar hyperparámetros, hay que dividir en tres conjuntos de datos: Training, Validation, Testing (60%, 20%, 20%). Se le recomienda confirmar con el profesor cuántos y cuáles hyperparámetros validar.
    1. Programación del diseño experimental. 
    1. Tablas de resultados, gráficos y conclusiones de los resultados, recomendando a su contratante cuál algoritmo utilizar, con qué configuración y por qué lo recomienda. 

# Preprocessing

First, we load and study the dataset. In this step, we look for missing values, outliers and any other information that may affect the model.

In [50]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

In [64]:
# loads file as a DataFrame
data = pd.read_csv('mnist_ALL.csv', header = 0, sep=';') # index_col = 0
print(data.head(5)) # shows first 5 registers to check if everything is loaded correctly

   label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      5    0    0    0    0    0    0    0    0    0  ...      0      0   
1      0    0    0    0    0    0    0    0    0    0  ...      0      0   
2      4    0    0    0    0    0    0    0    0    0  ...      0      0   
3      1    0    0    0    0    0    0    0    0    0  ...      0      0   
4      9    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  

[5 rows x 785 columns]


We can easily determine that the dataset has no missing values.

In [10]:
data.isnull().mean().sum()

0.0

We can also observe that there are no outliers, since all of the values for each variable are within the expected range. According to the client, the range of values for each of the variables is between 0 and 255.

In [43]:
for i in data.columns:
    if not data[i].between(0,255).all():
        print(f'{i}: {data[i].min()}, {data[i].max()}')

Finally, we can determine the different categories in which we are going to classify the data, along with the amount of entries for each category.

In [49]:
data.groupby('label').size()

label
0    6903
1    7877
2    6990
3    7141
4    6824
5    6313
6    6876
7    7293
8    6825
9    6958
dtype: int64

## Dataset Separation + Feature Selection

After studying the dataset, we have determined that the data meets all of the requirements for analysis with the selected classification models. 

Before proceeding with the model comparison and selection, we must first divide the dataset into train (validation) and test sets. Once the split is carried out, we can also filter some of the dataset's features, since not all of them necessarily provide valuable info to our model. 

In [65]:
# separate dataframe into X and Y for model
Y = pd.DataFrame(data['label'])
X = data.drop(columns=['label'])

# data is split into train/validation and test sets
x_train_val, x_test, y_train_val, y_test = train_test_split(X, Y, 
                                                    test_size=0.20, 
                                                    shuffle=True,
                                                    random_state=42 #seed
                                                    )

x_train_val.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
47339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
sel = VarianceThreshold(threshold=0.01)
sel.fit(x_train_val)  # fit finds the features with zero variance

print(f'Feature amount: {x_train_val.shape[1]}')
print(f'Features with a variance greater than 0.01: {sum(sel.get_support())}')

Feature amount: 784
Features with a variance greater than 0.01: 710


In [67]:
x_train_val = sel.transform(x_train_val)
x_test = sel.transform(x_test)

print(f'Train shape: {x_train_val.shape}')
print(f'Test shape: {x_test.shape}')

Train shape: (56000, 710)
Test shape: (14000, 710)


___

# Parte 2  -  Clustering  (50 puntos)
___

### Considere lo siguiente:
1. Conjunto de datos: tarjetas.csv
1. Este dataset corresponde a datos de tarjetas de crédito, a partir de los cuales se desea descubrir cuántos grupos (clusters) se podrían encontrar.
1. Proponga al menos dos algoritmos a utilizar
1. Proponga una métrica, la cual debe ser **interna**.
1. Debe presentar en este cuaderno:
    1. Pre-procesamiento de los datos, explicando las decisiones en cada caso. Observará que en este caso viene bastante preparado
    1. Para cada uno de los algoritmos seleccionados: 
        1. Explicación del algoritmo.
        1. Explicación de la implementación seleccionada y de sus parámetros.
    1. Explicación del diseño experimental por ejecutar. En este caso recuerde que es aprendizaje NO Supervisado. Además del númeoro de clusters, se le recomienda confirmar con el profesor cuántos y cuáles hyperparámetros validar. 
    1. Programación del diseño experimental. 
    1. Muestre en una tabla los resultados de la métrica.
    1. Grafique las métrica para ver el criterio del codo.
    1. ¿Cuál es el númermo de cluster que considera mejor describe el conjunto de datos? ¿Cómo defendería su recomendación ante su contratante? (Considere en su respuesta los algoritmos que utilizó)